In [150]:
import numpy as np
import pandas as pd
from datetime import datetime as dt
import itertools

%matplotlib inline

In [151]:
# Read data from .csv files 

loc = "voetbal_1956-20220309/"

raw_cards_file = pd.read_csv (loc + '_cards.csv')
raw_goals_file = pd.read_csv (loc + '_goals.csv')
raw_matches_file = pd.read_csv (loc + '_matches.csv')
raw_players_file = pd.read_csv (loc + '_players.csv')
raw_subs_file = pd.read_csv (loc + '_subs.csv')

In [152]:
def assist(value):
    if pd.notna(value):
        return 1
    else:
        return 0
# should be used to assign the assits (if value in assit is not null, then 
# return 0, else return 1) and then we will use the assits to predict which
# team will make a goal

In [155]:
filtered_assists = raw_goals_file.drop(columns=['new_score','type','player','minute','round','type'])
filtered_assists['assist'] = raw_goals_file['assist'].apply(assist)
filtered_assists

,Unnamed: 0,saison,home,away,team,assist
0,0,1956-1957,afc-ajax,bvc-amsterdam,afc-ajax,1
1,1,1956-1957,afc-ajax,bvc-amsterdam,bvc-amsterdam,0
2,2,1956-1957,afc-ajax,bvc-amsterdam,bvc-amsterdam,0
3,3,1956-1957,afc-ajax,bvc-amsterdam,afc-ajax,0
4,4,1956-1957,afc-ajax,bvc-amsterdam,bvc-amsterdam,0
...,...,...,...,...,...,...
60864,639,2021-2022,willem-ii,sc-cambuur,willem-ii,1
60865,640,2021-2022,willem-ii,sc-cambuur,sc-cambuur,0
60866,641,2021-2022,willem-ii,sparta-rotterdam,sparta-rotterdam,1
60867,642,2021-2022,willem-ii,sparta-rotterdam,sparta-rotterdam,1


In [154]:
raw_cards_file['round'] = pd.to_numeric(raw_cards_file['round'])
raw_cards_file['minute'] = pd.to_numeric(raw_cards_file['minute'])
no_nans_cards = raw_cards_file.dropna()
no_nans_cards

,Unnamed: 0,saison,round,home,away,team,player,minute,type
0,0,1962-1963,8,heracles-almelo,fc-twente,fc-twente,henk-bosveld,40.0,rode kaart
1,0,1971-1972,5,fc-groningen,afc-ajax,fc-groningen,hugo-hovenkamp,84.0,rode kaart
2,1,1971-1972,5,fc-groningen,afc-ajax,afc-ajax,piet-keizer,84.0,rode kaart
45,0,1973-1974,24,afc-ajax,az-alkmaar,az-alkmaar,jan-mulder_2,60.0,rode kaart
56,11,1973-1974,23,fc-twente,de-graafschap,de-graafschap,chris-de-vries,57.0,rode kaart
...,...,...,...,...,...,...,...,...,...
32287,685,2021-2022,25,willem-ii,sc-heerenveen,willem-ii,goerkem-saglam,71.0,gele kaart
32288,686,2021-2022,12,willem-ii,sparta-rotterdam,willem-ii,kwasi-wriedt,26.0,gele kaart
32289,687,2021-2022,12,willem-ii,sparta-rotterdam,sparta-rotterdam,dirk-abels,77.0,gele kaart
32290,688,2021-2022,12,willem-ii,sparta-rotterdam,sparta-rotterdam,adil-auassar,34.0,gele kaart
